In [1]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pandas as pd
import pickle

In [2]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [3]:
year = '2017'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [4]:
repickle=True

# Check if pickle exists, and don't re-create it if it does
picklename = 'pickles/'+str(year)+'_cutflow.pkl'
if os.path.isfile(picklename):
    repickle=False

In [5]:
# Load all files - this takes a while
if repickle:
    nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
    for n in range(1,nfiles+1):

        with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
            infiles = json.load(f)
    
        filename = '/myeosdir/vbf-category/outfiles/'+year+'_'+str(n)+'.coffea'
        if os.path.isfile(filename):
            out = util.load(filename)
            outsum.add(out)
        else:
            print('Missing file '+str(n),infiles.keys())
            #print("File " + filename + " is missing")
        
    scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
    outsum['cutflow'].scale(scale_lumi, 'dataset')
    cutflow = outsum['cutflow'].group('dataset', hist.Cat('process', 'Process'), pmap)

    outfile = open(picklename, 'wb')
    pickle.dump(cutflow, outfile, protocol=-1)
    outfile.close()

In [6]:
# Read the histogram from the pickle file
templates = pickle.load(open(picklename,'rb'))

In [7]:
cutflow = templates.sum('genflavor').integrate('region','signal')
cutflow

<Hist (process,cut) instance at 0x7f2075bdf0a0>

In [8]:
cutflow.values()

{('ZH',): array([2.41227002e+04, 2.41227002e+04, 2.41227002e+04, 1.07969582e+02,
        1.07969582e+02, 1.03299716e+02, 6.27966759e+01, 2.98563201e+01,
        2.25627656e+01, 1.91851559e+01, 1.67150935e+01, 6.42435792e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 ('WH',): array([2.47459912e+04, 2.47459912e+04, 2.47459912e+04, 1.02358134e+02,
        1.02358134e+02, 9.74804318e+01, 5.65834603e+01, 3.19033811e+01,
        2.01423922e+01, 1.67454151e+01, 1.30501774e+01, 3.99480873e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 ('ttH',): array([1.29382686e+04, 1.29382686e+04, 1.29382686e+04, 4.92626404e+02,
        4.92626404e+02, 4.19708001e+02, 1.47617577e+02, 4.96687165e+01,
        3.82192608e+01, 2.51782154e+01, 2.50686182e+01, 6.56587666e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 ('VBF',): array([9.50826122e+04, 9.50826122e+04, 9.50826122e+04, 1.02765541e+02,
        1.02765541e+02, 1.00015447e+02, 5.54427629e+01, 4.55900001e+

In [9]:
df1 = pd.DataFrame([])

df1['ggF'] = cutflow.values()[('ggF',)]
df1['VBF'] = cutflow.values()[('VBF',)]
df1['WH'] = cutflow.values()[('WH',)]
df1['ZH'] = cutflow.values()[('ZH',)]
df1['ttH'] = cutflow.values()[('ttH',)]

df1 = df1[3:-3].astype('int')
df1.index = ['Jet kinematics','Jet ID','Jet acceptance','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons','2 AK4 jets','DeepDoubleB']

df1.to_latex(buf=year+'/cutflow-sig.tex')

In [10]:
df2 = pd.DataFrame([])

df2['QCD'] = cutflow.values()[('QCD',)]
df2['Wjets'] = cutflow.values()[('Wjets',)]
df2['Zjets'] = cutflow.values()[('Zjets',)]
df2['VV'] = cutflow.values()[('VV',)]
df2['ttbar'] = cutflow.values()[('ttbar',)]
df2['singlet'] = cutflow.values()[('singlet',)]

df2 = df2[3:-3].astype('int')
df2.index = ['Jet kinematics','Jet ID','Jet acceptance','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons','2 AK4 jets','DeepDoubleB']

df2.to_latex(buf=year+'/cutflow-bkg.tex')